In [1]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

# input and output

In [2]:
###input
cut_off = 0.3
num = 50
num_process = 50
cut_off_path = f'../../../Results/not_lipid/top{num}_{cut_off}_re/'
yeast870_path =  '../../../Data/model/yeast-GEM902.yml'
miss_met_id_file = '../../../Data/model/miss_met_id.json'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
rxndb_total_info_to_model_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
yeast_genome_file = '../../../Data/Saccharomyces_cerevisiae.fasta'


## output
rxndb2model_compouns_protein_pair_df_file = '../../../Results/compound_protein/rxndb2model_compouns_protein_pair_df.csv'
rxndb2model_compouns_protein_pair_df_drop_file = '../../../Results/compound_protein/rxndb2model_compouns_protein_pair_df_drop.csv'

## rxndb2compound-protein-pair

In [3]:
rxndb_total_info_to_model = pd.read_csv(rxndb_total_info_to_model_path)
rxndb_total_info_to_model.head()

,NO,EC_number,templateID,rxn_smiles_basic,rxn_smiles_final,GPR,equation_dict,equation
0,rxn19747,1.14.13,MNXR180996_reverse,COc1cc(CC(=O)[O-])ccc1O.O>>COc1cc(C(C)=O)ccc1O...,COc1cc(CC(=O)[O-])ccc1O.O.NC(=O)c1ccc[n+]([C@@...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'sn_256': -1, 's_0803': -1, 's_1198': -1, 'sn...",sn_256 + s_0803 + s_1198 => sn_226 + s_12...
1,rxn19748,1.14.13,MNXR149799,CCCCCCCCCCCC(=O)[O-].O=O>>CCCCCCCCCC(O)CC(=O)[...,CCCCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'s_1065': -1, 's_1275': -1, 's_0794': -1, 's_...",s_1065 + s_1275 + s_0794 + s_1212 => sn_2...
2,rxn19749,1.14.13,MNXR149799,CCCCCCCCCC(=O)[O-].O=O>>CCCCCCCC(O)CC(=O)[O-].O,CCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@H]...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'s_0595': -1, 's_1275': -1, 's_0794': -1, 's_...",s_0595 + s_1275 + s_0794 + s_1212 => sn_1...
3,rxn19750,1.14.13,MNXR149799,CCCCCCCC(=O)[O-].O=O>>CCCCCC(O)CC(=O)[O-].O,CCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@H]2O...,(YBL098W) or (YNR053C) or (YCL059C) or (YHR176...,"{'s_1248': -1, 's_1275': -1, 's_0794': -1, 's_...",s_1248 + s_1275 + s_0794 + s_1212 => sn_2...
4,rxn19770,noec,MNXR114398,CCC(=[NH2+])C(=O)[O-].O>>CCC(=O)C(=O)[O-].[NH4+],CCC(=[NH2+])C(=O)[O-].O>>CCC(=O)C(=O)[O-].[NH4+],(nogene),"{'sn_414': -1, 's_0803': -1, 's_0178': 1, 's_0...",sn_414 + s_0803 => s_0178 + s_0419


In [4]:
rxndb_total_info_to_model['equation'].to_list()[2]

' s_0595 +  s_1275 +  s_0794 +  s_1212 =>  sn_197 +  s_0803 +  s_1207'

In [5]:
rxndb2model_compouns_protein_pair = {'NO':[],
                               'SMILES':[],
                               'gene':[],
                               }

for index,row in rxndb_total_info_to_model.iterrows():
    smiles_list = row['rxn_smiles_basic'].split('>>')[0].split('.')
    gene_list = set(row['GPR'].replace(')','').replace('(','').split(' or '))
    for smiles in smiles_list:
        for gene in gene_list:
            rxndb2model_compouns_protein_pair['NO'].append(row['NO'])
            rxndb2model_compouns_protein_pair['SMILES'].append(smiles)
            rxndb2model_compouns_protein_pair['gene'].append(gene)


In [6]:
rxndb2model_compouns_protein_pair_df = pd.DataFrame(rxndb2model_compouns_protein_pair)

In [7]:
rxndb2model_compouns_protein_pair_df_drop = rxndb2model_compouns_protein_pair_df.drop_duplicates(subset=['SMILES','gene'])

In [8]:
def fasta_to_df(fasta_name):
    data = []
    with open(fasta_name, 'r') as fasta:
        header = None
        sequence = []
        for line in fasta:
            if line.startswith('>'):
                if header is not None:
                    data.append((header, ''.join(sequence)))
                header = line.strip()[1:]
                sequence = []
            elif line.strip() != '':
                sequence.append(line.strip())

    if header is not None:
        data.append((header, ''.join(sequence)))

    df = pd.DataFrame(data, columns=['ID', 'Sequence'])

    return df

yeast_genome = fasta_to_df(yeast_genome_file)
yeast_genome.head(3)

def get_sequence(gene,yeast_genome):
    if gene in yeast_genome.ID.tolist():
        return yeast_genome.loc[yeast_genome['ID'] == gene]['Sequence'].values[0]
    return None

print(get_sequence('YDL204W',yeast_genome))

MNRNTTTNKNANLNNSRNANAPGEAGHQNKTGLIYWTNPSKSGASFAATLVSLLILRNVNVISVLLKIGYMVLFTSFAVELSTKVLFDKGVVSRFGMQESPDLVGVLKPHIDRELDRLPALEDRIRKLVFAHRTRNNFTIGVSLYFLHGLFAIFSMNTVLIMTTIFLYTVPLIYDRKQARIDRAIDRMKDLVIHRFHKNYNKVVEKTEPYIDKIIPPQTDEGSYSTSISNENKSSTSQRNKSGLSSSEFDNMNDTSASKSGKDSYSTSQYNRAEYPVSQNENIGTLKSGKQEIPTEKDFNNRHENFSKPDVKTYDPRTVDIEEELAAHQRELEQNLKDGDYNLVGSKEIPDPITVPAPTRHTTKPAESQSIPIKNNETLHKTTHGLKQKLQHA


In [9]:
rxndb2model_compouns_protein_pair_df_drop['Sequence'] = rxndb2model_compouns_protein_pair_df_drop['gene'].progress_apply(lambda x:get_sequence(x,yeast_genome))
rxndb2model_compouns_protein_pair_df_drop

100%|██████████| 193542/193542 [02:12<00:00, 1465.15it/s]
/home/wuke/anaconda3/envs/kcat/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,NO,SMILES,gene,Sequence
0,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YDR534C,MKLSSAFVLSAITVAALGESITTTITATKNGHVYTKTVTQDATFVW...
1,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YDR402C,MELLKLLCLILFLTLSYVAFAIIVPPLNFPKNIPTIPFYVVFLPVI...
2,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YBR298C,MKGLSSLINRKKDRNDSHLDEIENGVNATEFNSIEMEEQGKKSDFD...
3,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YBL098W,MSESVAIIGAGLVGCLAALAFSKEGYNVTLYDFRQDPRLDTTKNKN...
4,rxn19747,COc1cc(CC(=O)[O-])ccc1O,YOL114C,MTTLMGKFKLTGRSPLFVLQPMLHCKKQQFVEEAVRLISNKKIGKK...
...,...,...,...,...
748315,rxn1349,O=P([O-])([O-])OCC1OC(OC2(CO)OC(CO)C(O)C2O)C(O...,YEL029C,MPRLLATQSHVVHGYVGNKAATFPLQCLGWDVDCCNSVQFSNHTGY...
748316,rxn1349,O=P([O-])([O-])OCC1OC(OC2(CO)OC(CO)C(O)C2O)C(O...,YOR347C,MPESRLQRLANLKIGTPQQLRRTSIIGTIGPKTNSCEAITALRKAG...
748317,rxn1349,O=P([O-])([O-])OCC1OC(OC2(CO)OC(CO)C(O)C2O)C(O...,YBR020W,MTKSHSEEVIVPEFNSSAKELPRPLAEKCPSIIKKFISAYDAKPDF...
748318,rxn1349,O=P([O-])([O-])OCC1OC(OC2(CO)OC(CO)C(O)C2O)C(O...,YJR049C,MKENDMNNGVDKWVNEEDGRNDHHNNNNNLMKKAMMNNEQIDRTQD...


In [ ]:
print(rxndb2model_compouns_protein_pair_df_drop.shape)
rxndb2model_compouns_protein_pair_df_drop = rxndb2model_compouns_protein_pair_df_drop[~rxndb2model_compouns_protein_pair_df_drop['Sequence'].isna()]
print(rxndb2model_compouns_protein_pair_df_drop.shape)

(193542, 4)
(193520, 4)


In [11]:
rxndb2model_compouns_protein_pair_df_drop.to_csv(rxndb2model_compouns_protein_pair_df_drop_file,index=None)

In [12]:
rxndb2model_compouns_protein_pair_df.to_csv(rxndb2model_compouns_protein_pair_df_file,index=None)